In [ ]:
%matplotlib inline

# N400: Test using words

The N400 is a negative-going deflection first discovered in response to sentence-final words that were incongruent with their context. Because the N400 is typically observed using central electrodes, we are going to be using a pre-existing dataset known as [ERP CORE](https://erpinfo.org/erp-core) to examine how these data are typically processed. 

Since we're using a different dataset, the data are formatted differently, so the below code will look different from what we have been using. But some of the basic principles will remain the same.

In [ ]:
# general imports
import os
import numpy as np

# MNE-based imports
from mne import read_epochs, read_evokeds, write_evokeds, combine_evoked, grand_average
from mne import Epochs,Evoked,find_events
from mne.viz import plot_compare_evokeds 

ERP CORE presents their files in raw format for individuals to preprocess using their own methods. Because their paper originally uses MATLAB to process files, we are going to process them using MNE. The first step in this process is to preprocess data. I have done some of this in advance, so that we can see how it works, but we won't be doing the rest of it today (we'll actually all take part in that *fun* process together when we discuss analyses in more detail).

So instead, we are loading in some files. I will be having all groups look at one different participant first before we average them all together.

All participant files have been uploaded onto Canvas, and your assigned file is on the 'N400 File Assignment' spreadsheet. The file you want to download will be in the 'Epoched Files' folder. Download your assigned participant into the 'N400' folder directory. Your path in Windows Explorer should look like:  
C: / Users / eeg-user / eeg-notebooks / introeeg_labs / N400

In [ ]:
# loading in files
file_path = os.path.join('C:\\', 'Users', 'eeg-user', 'eeg-notebooks', 'introeeg_labs', 'N400')

subject = 2 # replace this with the subject number you have been assigned
filename = str(subject) + "_N400_epo.fif" # name of the file that was made
mmn_epoch_subj = os.path.join(file_path, filename)
epochs = read_epochs(mmn_epoch_subj)

These files are epoched data, meaning that they have already been segmented into the time periods immediately before and after a stimulus was presented. These are so we can make ERPs out of averaging them togoether. 

## Averaging conditions

In the N400 data used for this test, there are two conditions of interest: control and violation. They have been previously labeled in events as such, so we can start by averaging them.

In [ ]:
# averaging related and unrelated, but ONLY for the targets (not the primes)

related = epochs['target/related'].average()
unrelated = epochs['target/unrelated'].average()

In order to save these averages, we make an mne.Evoked object from the epochs that we have separated out (this is what averaging epochs does, as we did above).

We'll take a brief look at what it looks like for one participant. The N400 is measured from electrode CPz in the ERP CORE set, with the epoch set from 200 milliseconds before stimulus onset to 800 milliseconds after.

In [ ]:
evokeds = dict(related=related, unrelated=unrelated)
picks = ['CPz']
plot_compare_evokeds(evokeds, picks=picks)

Their group average, with 39 participants, looks like this:

![N400 from ERPCORE](n400_erpcore.png)

So let's make our class average with a subset of their data! First, we have to save the file you just made.

In [ ]:
# save evokeds file
fname = str(subject) + "_N400_evo-ave.fif"
write_evokeds(fname, list(evokeds.values()), overwrite=True)

Now take this evo-ave file that you've generated, and upload it into the 'Finished Files' folder.

## Averaging across participants

We've now each generated our own evoked files, and uploaded them into the 'Finished Files' folder. Now download the files that everyone else has made and put them into the same folder you put your first file (and where the evo-ave file you just made should be saved). We are now going to do the same thing we did with one participant, but across all of the participants in our class set!

In [ ]:
# using mne.grand_average() to combine subjects

file_path = os.path.join('C:\\', 'Users', 'nestea', 'eeg-notebooks', 'introeeg_labs', 'N400')
os.chdir(file_path)

subjects = [2] # this should be replaced with the numbers of the subjects that we are using!
files = []
evoked_subs = {}
conditions = ['related', 'unrelated'] # list out conditions
picks=['CPz']

# make file list
for subj in subjects:
    fname = str(subj) + "_N400_evo-ave.fif"
    files.append(fname)

for idx, c in enumerate(conditions):
    evoked_subs[c] = [read_evokeds(fname)[idx] for fname in files]

r_grand_average = grand_average(evoked_subs['related'])
u_grand_average = grand_average(evoked_subs['unrelated'])
grand = dict(related=r_grand_average, unrelated=u_grand_average)
plot_compare_evokeds(evoked_subs, picks=picks)